In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model('res152animalclass2.h5')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

class_names = ['antelope', 'bear', 'beaver', 'bee', 'bison', 'blackbird', 'buffalo', 'butterfly', 'camel', 'cat',
               'cheetah', 'chimpanzee', 'chinchilla', 'cow', 'crab', 'crocodile', 'deer', 'dog', 'dolphin', 'donkey',
               'duck', 'eagle', 'elephant', 'falcon', 'ferret', 'flamingo', 'fox', 'frog', 'giraffe', 'goat', 'goose',
               'gorilla', 'grasshopper', 'hawk', 'hedgehog', 'hippopotamus', 'hyena', 'iguana', 'jaguar', 'kangaroo',
               'koala', 'lemur', 'leopard', 'lizard', 'lynx', 'mole', 'mongoose', 'ostrich', 'otter', 'owl', 'panda',
               'peacock', 'penguin', 'porcupine', 'raccoon', 'seal', 'sheep', 'snail', 'snake', 'spider', 'squid',
               'walrus', 'whale', 'wolf']

def rescale(image):
    return image / 255.0

def normalize_image(img):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = (img - mean) / std
    return img

def preprocess_image(image_path, target_shape=(224, 224)):
    image = cv2.imread(image_path)
    if image is not None:
        image_resized = cv2.resize(image, target_shape)
        image_rescaled = rescale(image_resized)
        image_normalized = normalize_image(image_rescaled)
        return np.expand_dims(image_normalized, axis=0)
    return None

def predict_animal(image_path):
    processed_image = preprocess_image(image_path)
    predictions = model.predict(processed_image)
    return predictions[0]

def browse_file():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        img = Image.open(file_path)
        img.thumbnail((400, 400))
        img = ImageTk.PhotoImage(img)
        panel.configure(image=img)
        panel.image = img

        for item in result_tree.get_children():
            result_tree.delete(item)

        predictions = predict_animal(file_path)
        sorted_predictions = sorted(zip(class_names, predictions), key=lambda x: x[1], reverse=True)

        for class_name, confidence in sorted_predictions:
            result_tree.insert('', 'end', values=(class_name, f"{confidence:.2%}"))

        if sorted_predictions:
            top_prediction = sorted_predictions[0]
            top_result_label.configure(text=f"This animal is a {top_prediction[0]} with confidence {top_prediction[1]:.2%}")
        else:
            top_result_label.configure(text="No predictions available.")

app = tk.Tk()
app.title("Animal Classifier")
app.geometry("1000x1000")
app.configure(bg="#FFFAF0")

header_label = tk.Label(app, text="Animal Classifier", font=("Arial", 24, "bold"), bg="#FFFAF0", fg="#2F4F4F")
header_label.pack(pady=10)

panel = tk.Label(app, bg="#FFFAF0")
panel.pack(pady=20)

browse_button = tk.Button(app, text="Browse Image", command=browse_file, font=("Arial", 14), bg="#20B2AA", fg="white")
browse_button.pack(pady=10)

result_tree = ttk.Treeview(app, columns=("Class", "Confidence"), show="headings", height=15)
result_tree.heading("Class", text="Class")
result_tree.heading("Confidence", text="Confidence")
result_tree.column("Class", width=200, anchor="center")
result_tree.column("Confidence", width=200, anchor="center")
result_tree.pack(pady=20)

top_result_label = tk.Label(app, text="", font=("Arial", 14), bg="#FFFAF0", fg="#2F4F4F")
top_result_label.place(x=150, y=750)
top_result_label.pack(pady=20)

app.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
